In [1]:
# ============================================
# 📘 Urdu Sentiment Analysis using BERT-base
# Dataset: IMDB 50K Urdu-translated Reviews
# ============================================

!pip install transformers datasets kagglehub torch torchvision torchaudio scikit-learn tqdm

# --------------------------------------------
# Step 1: Download dataset from KaggleHub
# --------------------------------------------
import kagglehub

path = kagglehub.dataset_download("akkefa/imdb-dataset-of-50k-movie-translated-urdu-reviews")
print("📁 Dataset path:", path)

  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached nvidia_nccl_cu12-2.27.3-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.0 kB)
  Using cached triton-3.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl (887.9 MB)
Using cached nvidia_nccl_cu12-2.27.3-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (322.4 MB)
Using cached triton-3.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (155.6 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.5.0
    Uninstalling triton-3.5.0:
      Successfully uninstalled triton-3.5.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.27.5
    Uninstalling nvidia-nccl-cu12-2.27.5:
      Successfully uninstalled nvidia-nccl-cu12-2.27.5
  Attempting uninstall: torch
    Found existing installation: torch 2.9.0
  

# Simple BERT base

In [2]:
import os
import pandas as pd

# List all files in the downloaded path
print("📂 Files inside dataset folder:")
print(os.listdir(path))

# Try to auto-detect the CSV file
for file in os.listdir(path):
    if file.endswith(".csv"):
        csv_path = os.path.join(path, file)
        break

print("✅ Detected CSV file:", csv_path)

# Now read it
df = pd.read_csv(csv_path)
print("✅ Dataset Loaded Successfully!")
print(df.head())


📂 Files inside dataset folder:
['imdb_urdu_reviews_test.csv', 'imdb_urdu_reviews_train.csv']
✅ Detected CSV file: /root/.cache/kagglehub/datasets/akkefa/imdb-dataset-of-50k-movie-translated-urdu-reviews/versions/6/imdb_urdu_reviews_test.csv
✅ Dataset Loaded Successfully!
                                              review sentiment
0  یہ بے گھر خواتین کے بارے میں ایک دستاویزی فلم ...  negative
1  بالکل بھی اچھ ،ی کام نہیں کیا گیا ، پوری فلم ص...  negative
2  یہ عجیب بات ہے کہ کچھ لوگوں کا کیا حشر ہوتا ہے...  negative
3  اور یہ خاص طور پر وکیلوں اور پولیس اہلکاروں کے...  positive
4  پہلے ، ایک وضاحت: میری سرخی کے باوجود ، میں اس...  positive


In [3]:
# --------------------------------------------
# Step 3: Data Preprocessing
# --------------------------------------------
import re

def clean_urdu_text(text):
    text = re.sub(r'[^\u0600-\u06FF\s]', '', str(text))  # keep only Urdu chars
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['review'] = df['review'].apply(clean_urdu_text)
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

print("🔤 Sample Cleaned Review:")
print(df['review'].iloc[0])

🔤 Sample Cleaned Review:
یہ بے گھر خواتین کے بارے میں ایک دستاویزی فلم ہے۔ اس لحاظ سے یہ دلچسپ بات ہے کہ اس کی توجہ ان خواتین پر مرکوز ہے جو معاشرتی طور پر مصروف ہیں ملازمتیں اور دیرپا دوستی لیکن ایسی صورتحال میں ہیں جہاں وہ رہائش کا متحمل نہیں ہوسکتے ہیں۔ مجھے کچھ خواتین دلچسپ معلوم ہوئیں ، لیکن اس میں بہت کم توجہ دی گئی یا کہانی میں ترقی سمت اور ترمیم میری توجہ برقرار رکھنے میں ناکام رہی۔ یقینا ان خواتین کی کہانیوں میں بھی اختلافات تھے ، لیکن پیغام بنیادی طور پر ایک ہی تھا اور زیادہ سے زیادہ گہرائی میں ان میں سے کسی پر توجہ مرکوز کرکے بتایا جاسکتا تھا۔ میں نے اسے فلم کے اختتام تک پہنچا دیا ، لیکن یہ ایک ایسی فلم تھی بلکہ بورنگ سفر۔


In [4]:
# --------------------------------------------
# Step 4: Split Dataset (Train / Val / Test)
# --------------------------------------------
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"📊 Dataset split -> Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

📊 Dataset split -> Train: 8000, Val: 1000, Test: 1000


In [5]:
# Step 5: Tokenization (BERT-base)
# --------------------------------------------
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True, max_length=128)

# Convert into Hugging Face dataset format
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[['review', 'label']])
val_dataset = Dataset.from_pandas(val_df[['review', 'label']])
test_dataset = Dataset.from_pandas(test_df[['review', 'label']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["review"])
val_dataset = val_dataset.remove_columns(["review"])
test_dataset = test_dataset.remove_columns(["review"])

train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
# Step 6: Model Setup (BERT-base)
# --------------------------------------------
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# --------------------------------------------
# Step 7: Define Metrics Function
# --------------------------------------------
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    rec = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1
    }

In [8]:
# -------------------------
# Robust TrainingArguments creation
# -------------------------
import transformers
from transformers import TrainingArguments, Trainer
import inspect
import traceback
import time

print("Transformers version:", transformers.__version__)
print("TrainingArguments signature:")
print(inspect.signature(TrainingArguments.__init__))

# Common args we want to pass (modern defaults)
modern_args = dict(
    output_dir="./results_bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

# Minimal args for older versions (fallback)
fallback_args = dict(
    output_dir="./results_bert",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir="./logs",
)

# Try modern first, then fallback
training_args = None
used_mode = None
try:
    training_args = TrainingArguments(**modern_args)
    used_mode = "modern"
    print("✅ Created TrainingArguments with modern args.")
except TypeError as e:
    print("⚠️ Modern args failed:", e)
    print("Attempting fallback args...")
    try:
        training_args = TrainingArguments(**fallback_args)
        used_mode = "fallback"
        print("✅ Created TrainingArguments with fallback args.")
    except Exception as e2:
        print("❌ Fallback also failed. Showing full error:")
        traceback.print_exc()
        raise RuntimeError("Could not create TrainingArguments with either modern or fallback args. Consider upgrading transformers with `!pip install -U transformers` and restart the runtime.") from e2

# Print chosen args summary for transparency
print("\n--- TrainingArguments summary (selected mode =", used_mode, ") ---")
for k, v in (modern_args.items() if used_mode=="modern" else fallback_args.items()):
    print(f"{k}: {v}")

# Now create the Trainer (example; replace datasets/model/compute_metrics with your objects)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics,
# )
#
# If Trainer creation later errors, print its signature for debugging:
print("\nTrainer signature:")
print(inspect.signature(Trainer.__init__))


Transformers version: 4.57.1
TrainingArguments signature:
(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: float = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict[str, Any], str] = 

In [9]:
from transformers import Trainer

# Step 9: Trainer Creation
# --------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)




/tmp/ipython-input-359536104.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
# --------------------------------------------
# Step 9: Train the Model (and Measure Time)
# --------------------------------------------
start_time = time.time()
trainer.train()
end_time = time.time()
training_time = round((end_time - start_time) / 60, 2)

print(f"⏱️ Total Training Time: {training_time} minutes")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mshoaibb707 (mshoaibb707-aror-university-sukkur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.703100
1000,0.697000
1500,0.695500


⏱️ Total Training Time: 23.96 minutes


In [11]:

# Step 10: Evaluate on Test Set
# --------------------------------------------
test_results = trainer.evaluate(test_dataset)
print("📊 Test Results:")
print(test_results)



📊 Test Results:
{'eval_loss': 0.6930848360061646, 'eval_accuracy': 0.508, 'eval_precision': 0.508, 'eval_recall': 1.0, 'eval_f1': 0.6737400530503979, 'eval_runtime': 6.4614, 'eval_samples_per_second': 154.766, 'eval_steps_per_second': 9.75, 'epoch': 3.0}


In [12]:
# --------------------------------------------
# Step 11: Summarize Results
# --------------------------------------------
summary = {
    "Model": "BERT-base",
    "Accuracy": round(test_results['eval_accuracy'], 4),
    "Precision": round(test_results['eval_precision'], 4),
    "Recall": round(test_results['eval_recall'], 4),
    "F1-score": round(test_results['eval_f1'], 4),
    "Training Time (mins)": training_time,
    "Comments": "Baseline English BERT fine-tuned on Urdu-translated IMDB reviews"
}

summary_df = pd.DataFrame([summary])
print("\n✅ Model Performance Summary:")
print(summary_df)

# Save results
summary_df.to_csv("bert_urdu_results.csv", index=False)



✅ Model Performance Summary:
       Model  Accuracy  Precision  Recall  F1-score  Training Time (mins)  \
0  BERT-base     0.508      0.508     1.0    0.6737                 23.96   

                                            Comments  
0  Baseline English BERT fine-tuned on Urdu-trans...  


In [14]:
# Step 11: Evaluate Model
# --------------------------------------------
eval_results = trainer.evaluate()
print("📊 Evaluation Metrics:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


📊 Evaluation Metrics:
eval_loss: 0.6930859088897705
eval_accuracy: 0.508
eval_precision: 0.508
eval_recall: 1.0
eval_f1: 0.6737400530503979
eval_runtime: 7.8501
eval_samples_per_second: 127.387
eval_steps_per_second: 8.025
epoch: 3.0
